In [5]:
import sys
sys.path.append('/Users/kuba/code/aic/mill_python')
import mil_ptevrthng as mil_pt
import mil as mil_np
import utils

import time

import torch

In [6]:
# Set model
input_len = 1000
n_neurons1 = 8
learning_rate = 1e-2
weight_decay = 1e-6

# # 1 - layer
# prepNN1 = torch.nn.Sequential(
#     torch.nn.Linear(input_len, n_neurons1, bias = True),
#     torch.nn.ReLU(),
# )

# afterNN1 = torch.nn.Sequential(
#     torch.nn.Linear(n_neurons1, 1),
#     torch.nn.Tanh()
# )

# model_np = torch.nn.Sequential(
#     mil_np.BagModel(prepNN1, afterNN1, torch.mean)
# ).double()

# model_pt = torch.nn.Sequential(
#     mil_pt.BagModel(prepNN1, afterNN1, torch.mean)
# ).double()

# 2 - layer
prepNN1 = torch.nn.Sequential(
    torch.nn.Linear(input_len, n_neurons1, bias = True),
    torch.nn.ReLU(),
)

afterNN1 = torch.nn.Sequential(
    torch.nn.Identity()
)

prepNN2 = torch.nn.Sequential(
    torch.nn.Linear(n_neurons1, n_neurons1, bias = True),
    torch.nn.ReLU(),
)

afterNN2 = torch.nn.Sequential(
    torch.nn.Linear(n_neurons1, 1),
    torch.nn.Tanh()
)

# Model and loss function
model_np = torch.nn.Sequential(
    mil_np.BagModel(prepNN1, afterNN1, torch.mean),
    mil_np.BagModel(prepNN2, afterNN2, torch.mean)
).double()

model_pt = torch.nn.Sequential(
    mil_pt.BagModel(prepNN1, afterNN1, torch.mean),
    mil_pt.BagModel(prepNN2, afterNN2, torch.mean)
).double()

criterion = mil_pt.MyHingeLoss()
optimizer_np = torch.optim.Adam(model_np.parameters(), lr = learning_rate, weight_decay = weight_decay)
optimizer_pt = torch.optim.Adam(model_pt.parameters(), lr = learning_rate, weight_decay = weight_decay)

In [7]:
# Load parameters from numpy model to pytorch model
state_dict = model_np.state_dict()
model_pt.load_state_dict(state_dict, strict = True)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [8]:
# Create random data
data = torch.rand((8,1000)).double()
ids = torch.Tensor([[1,1,1, 2,2, 3,3,3],[1, 2, 2, 3, 4, 5, 6, 6]]).long()
print(data)

tensor([[0.8236, 0.6470, 0.0806,  ..., 0.6529, 0.9353, 0.6031],
        [0.0095, 0.5546, 0.1555,  ..., 0.8914, 0.0678, 0.4612],
        [0.7384, 0.8318, 0.9265,  ..., 0.5492, 0.8308, 0.8648],
        ...,
        [0.8785, 0.9104, 0.4425,  ..., 0.4830, 0.6274, 0.1186],
        [0.6708, 0.1796, 0.6756,  ..., 0.2043, 0.0456, 0.2363],
        [0.6077, 0.2957, 0.7430,  ..., 0.2462, 0.6977, 0.1317]],
       dtype=torch.float64)


In [9]:
start = time.time()
model_np((data,ids))
print('Numpy version | Elapsed time: {}'.format(time.time() - start))

Numpy version | Elapsed time: 0.00396418571472168


In [10]:
start = time.time()
model_pt((data,ids))
print('PyTorch version | Elapsed time: {}'.format(time.time() - start))

PyTorch version | Elapsed time: 0.004330158233642578


In [14]:
import cProfile

In [17]:
cProfile.run('model_pt((data,ids))')

         220 function calls (203 primitive calls) in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 activation.py:307(forward)
        2    0.000    0.000    0.000    0.000 activation.py:97(forward)
      5/1    0.000    0.000    0.004    0.004 container.py:90(forward)
        3    0.000    0.000    0.001    0.000 functional.py:1390(linear)
        2    0.000    0.000    0.000    0.000 functional.py:394(unique)
        2    0.000    0.000    0.000    0.000 functional.py:932(relu)
        1    0.000    0.000    0.000    0.000 linear.py:31(forward)
        3    0.000    0.000    0.001    0.000 linear.py:90(forward)
        2    0.001    0.001    0.004    0.002 mil_ptevrthng.py:27(forward)
        2    0.000    0.000    0.000    0.000 mil_ptevrthng.py:46(<listcomp>)
     14/1    0.000    0.000    0.004    0.0